In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol

using StaticArrays
using Rotations
using Distances
using LinearAlgebra
using CairoMakie
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [3]:
template_radii = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_radii"]
radii = [template_radii; template_radii; template_radii; template_radii]
template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_centers"]
colors = [
    [MorphoMolNotebooks.STANDARD_COLORS[1] for _ in 1:length(template_radii)]; 
    [MorphoMolNotebooks.STANDARD_COLORS[2] for _ in 1:length(template_radii)]; 
    [MorphoMolNotebooks.STANDARD_COLORS[3] for _ in 1:length(template_radii)];
    [MorphoMolNotebooks.STANDARD_COLORS[4] for _ in 1:length(template_radii)];
    ]

# # Consecutive assembly
# R0 = RotMatrix(@SMatrix[1.000000  0.000000  0.000000; 0.000000  1.000000  0.000000; 0.000000  0.000000  1.000000])
# T0 = @SVector[0.00000, 0.00000, 0.00000]

# R1 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000])
# T1 = @SVector[-69.28043, 195.91352, -49.35000]

# R2 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000])
# T2 = @SVector[-61.30589, 104.11829, -47.94000]

# R3 = RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000])
# T3 = @SVector[-19.48193, 22.01644, -46.53000]

# consecutive = Vector{Float64}([])
# for (R,T) in zip([log(R1), log(R2), log(R3)], [T1, T2, T3])
#     consecutive = [consecutive; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
# end
# points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization_per_mol(consecutive, template_centers)...))]
# MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/3su_test/exp_1")

# Two top one bottom
R1 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000 1.000000])
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000 1.000000])
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = RotMatrix(@SMatrix[0.52145615 0.85327808 0.000000; -0.85327808 0.52145615 -0.000000; 0.000000 0.000000 1.000000])
T3 = @SVector[-63.89221, 227.07555, -26.79]

R4 = RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384 0.803441 0.000000; 0.000000 0.000000 1.000000])
T4 = @SVector[-67.99970, 135.02619, -25.38000]

two_top_one_bottom = Vector{Float64}([])
for (R,T) in zip([log(R1), log(R2), log(R3), log(R4)], [T1, T2, T3, T4])
    two_top_one_bottom = [two_top_one_bottom; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end
points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization_per_mol(two_top_one_bottom, template_centers)...))]
MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/4su_assembly/exp_1")


In [3]:
folder = "../../Data/hpc_out/3_6r7m/3/"
mol_type = "6r7m"
algo = "rwm"
energy = "ma"
folder = "../../Data/hpc_out/$(algo)_$(energy)_3_$(mol_type)/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end
            
            id = parse(Int, split(file, separator)[id_index])
            template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]

            mindex = argmin(output["Es"])
            simulated_assembly_state = output["states"][mindex];

            theta_string = ""
            permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
            for (name, experimental_state) in [("consecutive", consecutive), ("two_top_one_bottom", two_top_one_bottom), ("two_bottom_one_top", two_bottom_one_top)]
                dists = Vector{Float64}([])
                for perm in permutations
                    push!(dists, MorphoMol.Utilities.sum_of_permutation(template_centers, template_centers, simulated_assembly_state, experimental_state, [1, 2, 3], perm))
                end
                theta_string = theta_string * "$(name): $(minimum(dists)) |"
            end
            string = "$(id): $(input["T"]) | $(input["overlap_slope"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta_string)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
for elem in evaluation_strings
    println(elem)
end

Minimal configuration id: 58 | Average acceptance rate: 0.4721905426867306
1: 3.1 | 1.2 | 0.8723383 | 13144.112295431973, 13116.099453848048, 13144.112295495264 | consecutive: 563.4501167799116 |two_top_one_bottom: 567.9544162313932 |two_bottom_one_top: 567.9543888589042 |
2: 3.1 | 1.2 | 0.39083877 | 13144.112295840207, 13069.346709219064, 13102.329274407079 | consecutive: 332.5236425050354 |two_top_one_bottom: 337.02794195651717 |two_bottom_one_top: 337.02791458402817 |
3: 3.1 | 1.2 | 0.32839176 | 13144.112295762563, 13054.25573583572, 13144.112295333796 | consecutive: 513.5768597566463 |two_top_one_bottom: 518.081159208128 |two_bottom_one_top: 518.081131835639 |
4: 3.1 | 1.2 | 0.62906015 | 13144.112295727591, 13080.6259446557, 13097.948953392588 | consecutive: 91.69152747837082 |two_top_one_bottom: 96.19582692985256 |two_bottom_one_top: 96.19579955736354 |
5: 3.1 | 1.2 | 0.7693088 | 13144.112295551067, 13105.077578074002, 13144.11229537193 | consecutive: 492.002373390407 |two_top_one

In [4]:
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            mindex = argmin(output["Es"])
            state = output["states"][mindex]
            id = parse(Int, split(file, "_")[id_index])
            radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
            MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
        catch e
            println(e)
            continue
        end
    end
end

In [ ]:
function sum_of_permutation(template_centers_a::Matrix{Float64}, template_centers_b::Matrix{Float64}, state_a::Vector{Float64}, state_b::Vector{Float64}, perm_a::Vector{Int}, perm_b::Vector{Int})
    n = size(template_centers_a)[2]
    d1 = sum(MorphoMol.Utilities.get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[1,2]], perm_b[[1,2]])) / n
    d2 = sum(MorphoMol.Utilities.get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[2,3]], perm_b[[2,3]])) / n
    d3 = sum(MorphoMol.Utilities.get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[1,3]], perm_b[[1,3]])) / n
    println("d1: $d1 | d2: $d2 | d3: $d3")
    (d1 + d2 + d3)/3.0
end

sum_of_permutation (generic function with 1 method)

In [9]:
#file = "../SolSim.jl_dev/io/hpc/all_data/three_monomers/protor_oj_0_0_os_0_85/6r7m/5/75_6r7m_protor_oj_0_0_os_0_85.jld2"
file = "../../Data/hpc_out/3_6r7m/3/39_rwm_ma_3_6r7m.jld2"
@load file input output
n_mol = input["n_mol"]
n_atoms_per_mol = length(input["template_radii"])

mindex = argmin(output["Es"])
simulated_assembly_state = output["states"][mindex];
sanity_check_state = deepcopy(simulated_assembly_state) 
sanity_check_state[1] += 0.1

points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization_per_mol(simulated_assembly_state, input["template_centers"])...))]
radii = [input["template_radii"]; input["template_radii"]; input["template_radii"]];
colors = [MorphoMolNotebooks.STANDARD_COLORS[j] for j in 1:n_mol for i in 1:n_atoms_per_mol]
MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/3su_test/sa")

permutations = [[1,2,3], [1,3,2], [2,1,3], [2,3,1], [3,1,2], [3,2,1]]
for (name, experimental_state) in [("sanity_check_state", sanity_check_state)]
#for (name, experimental_state) in [("consecutive", consecutive), ("two_top_one_bottom", two_top_one_bottom), ("two_bottom_one_top", two_bottom_one_top), ("simulated_assembly_state", simulated_assembly_state), ("sanity_check_state", sanity_check_state)]
    dists = Vector{Float64}([])
    for perm in permutations
        d = sum_of_permutation(template_centers, template_centers, simulated_assembly_state, experimental_state, [1, 2, 3], perm)
        println("$(perm) | $(d)")
        push!(dists, d)
    end
    println("$(name) | $(minimum(dists))\n")
end

d1: 3.8984406210693434 | d2: 0.0 | d3: 8.30646919807704
[1, 2, 3] | 4.0683032730487945
d1: 424.46508693273347 | d2: 0.0 | d3: 428.87311550974124
[1, 3, 2] | 284.4460674808249
d1: 3.8984406210693434 | d2: 347.76008347720386 | d3: 356.066552675281
[2, 1, 3] | 235.9083589245181
d1: 76.70500345552963 | d2: 347.76008347720386 | d3: 428.87311550974124
[2, 3, 1] | 284.4460674808249
d1: 424.46508693273347 | d2: 72.8065628344603 | d3: 356.066552675281
[3, 1, 2] | 284.4460674808249
d1: 76.70500345552963 | d2: 72.8065628344603 | d3: 8.30646919807704
[3, 2, 1] | 52.60601182935566
sanity_check_state | 4.0683032730487945



In [9]:
42.97171653296904 + 70.72005462595087 + 96.06107300564682

209.75284416456674